In [2]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup

# Planning

### What data do we have?
- Everyone's picks and weights for n weeks
- The winners of every game

### League average pick
- Calculate a league average pick: an "expected estimate" of what the rest of the league will pick
    - Expected that the league average pick is closer to 0 than the actual score
    - Always positive -> just in the direction of the most picked team
    - Ranges from 0 - 16 
    - Closer to 0: league is pretty split
    - Closer to 16: league favoring one team
    
### League average Score
- Calculate a league average score: The actual average score for each game in the league
    - Ranges from 0 - 16
    - This is just the average number of points that the league gained
    - 

In [36]:
def html_to_df(week):
    filepath = f"data/week{week}.html"

    
    # na, correct, incorrect
    # week, game, correct points, incorrect points
    data = []

    with open(filepath, "r") as file:
        contents = file.read()
        picks_length = 16 # need to change for other weeks
        soup = BeautifulSoup(contents)

        rows = soup.find_all("tr", attrs={"class": "bg2"})
        person = 0
        for row in rows:
            picks = row.find_all("td", attrs={"class": ["na", "correct", "incorrect"]})
            if len(picks) != picks_length:
                continue
            i = 0
            for pick in picks:
                pick_class = pick["class"][0]
                if pick_class == "na":
                    #data.append([week, i, person, 0, 0])
                    i += 1
                    continue

                pick_text = pick.text
                pts = pick_text[pick_text.index("(")+1 : pick_text.index(")")]
                
                if pick_class == "correct":
                    data.append([week, i, person, int(pts), 0])
                else:
                    data.append([week, i, person, 0, int(pts)])

                i += 1
            person += 1
    
    df = pd.DataFrame(data, columns=["week", "game", "person", "correct_pts", "incorrect_pts"])

    return df
            
raw_data = html_to_df(3)

In [37]:
grouped = raw_data[["game", "correct_pts", "incorrect_pts"]].groupby(["game"])
res = grouped.sum()
res["count"] = grouped.game.count()
res.head(16)

,correct_pts,incorrect_pts,count
game,,,
0,201,68,33
1,280,49,31
2,70,296,31
3,317,24,31
4,282,40,31
5,222,60,31
6,11,408,31
7,60,173,31
8,66,149,31
